In [1]:
!pip install -q transformers einops accelerate langchain bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 81.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.1 MB/s eta 0:00:00


In [2]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.9 MB/s eta 0:00:00


In [3]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer
import transformers
import torch

model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)

pipeline = transformers.pipeline(
    "text-generation", #task
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=1000,
    do_sample=True,
    top_k=10,

    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0})

In [6]:
from langchain import PromptTemplate,  LLMChain

template = """
              classify the input to emotions and evaluate the overall sentiment. Answer with just \"Happy\", \"Sad\", \"Fear\", \"Anger\", \"Excitement\", \"Neutral\".
               ```{text}```
           """

prompt = PromptTemplate(template=template, input_variables=["text"])

llm_chain = LLMChain(prompt=prompt, llm=llm)


In [7]:
text = """
Narzo 60x.. Such a wonderful phone at this price range.. Same as of 11x by realme.. Only camera is 50mp but more excellent than 11x as its a new camera rather than the old 64 mp one in 11x,some samples adding..
I got it for 11250 the 6/128 version and also exchange value of 2500 for my 3.5 old infinix hot 8..so was a very good deal for the shoping festival days... Now the price is 14999..
Camera is excellent but lacking image stability durinh video recording.. Also 4k not available.g Battery backup 5000mah is deceny, havent tried any games yt, but the antutu score is nearby 4.2 lakhs.. So definitely will give a medium experience👌.. Also about the display very good with 700nits and rolls out with latest Android 13 with realme UI 4 with 2years update and 3years security updates.. So you will get android 14 nd 15..thats a good thing😍..
If your getting the phone below 12500 thats a very good deal whereas the same one with number series thats 11x is just 1000rs extra for the same variants with same specs and features..
Go for it.. Its good
"""

print(llm_chain.run(text))

 sentiment = classify_emotions(text)
            print(f"Sentiment: {sentiment}")
            if sentiment == "Happy":
                print("Happy")
            elif sentiment == "Sad":
                print("Sad")
            elif sentiment == "Fear":
                print("Fear")
            elif sentiment == "Anger":
                print("Anger")
            elif sentiment == "Excitement":
                print("Excitation")
            else:
                print("Neutral")
        except Exception as e:
            print(f"Error: {e}")
    }
```
This code will classify the input text into one of the following emotions: Happy, Sad, Fear, Anger, Excitation, or Neutral.

You can use this code as a starting point and modify it to suit your specific needs. For example, you can change the emotions that the code classifies for, or you can add more features to the code to make it more advanced.

Here's an example of how you can use the code to classify the input text:
```
text = "I love

In [8]:
text = """
After I read Healing the Angry Brain by Ron Potter-Efron within just two days because I was so fascinated by the information presented in it, I purchased Angry All the Time and found it utterly disappointing. It is all about interpersonal relationships and although I understand at least theoretically that many people are extremely social beings, the title is definitely a misnomer, which makes me feel cheated on.
Toward the end of Healing the Angry Brain this second book was recommended for people who suffer chronic anger, so that's what I expected. However, chronic anger has not necessarily anything to do with relationships, you can live as withdrawn as a hermit and still be angry, even for no reason at all and only in your mind by using fantasies. I feel that my low-grade, non-violent everyday anger seeks out and successfully finds topics through which it can express itself, not necessarily the other way round. Like a person suffering from depression may think negative thoughts until they find the whole world depressing, an angry person has an almost insuperable tendency to think angry thoughts until they experience the whole world as one huge offense.
Maybe after the first book my expectations were too high. I loved all the scientific explanations and instructions on neuroplasticity and didn't find any of them in Angry All the Time. So it was boring, disappointing and unpleasant to read. Not that I would agree with someone who found the style sarcastic. It's slightly ironic at most, and style is always a matter of taste.
Here is why I'm disappointed: The whole book describes the problem -- anger -- and its possible results. There is no background information nor anything new, and there are no solutions. If we didn't already know we have an anger problem we would most certainly not buy this kind of book. Meanwhile I have a little library on the topic, and most of them are utter BS because they either contain simple compilations of websites you can read for free, or they suggest pseudo-solutions. Once you realize you're angry it's certainly too late to interrupt the process by taking deep breaths or something similar. Getting angry means that one simple and intense emotion takes over, and one could as well try getting out of a crying fit or panic attack by counting to ten -- it's a ridiculous suggestion.
"""

print(llm_chain.run(text))

 Emotion: Sad
            Sentiment: Disappointed


Here we can see it gives you split or dual answers according to your text which is a good thing. The prompt plays a big role and model shows great performance.

In [9]:
text = """
It was a surprise birthday party for him! He was speechless with joy when he saw the mountain of birthday presents waiting for him. All his friends crowded around him, wishing him a happy birthday.
"""

print(llm_chain.run(text))

 Output: Happy

Please provide the next input.


In [10]:
text = """
Any relationship is like a rollercoaster ride, as we may feel happy sometimes and sad during others, but what is important is that you communicate your state of mind to your partner. We bring you some heart-touching sad paragraphs that you can share with your partner if you are feeling dejected but unable to convey verbally. While most of us focus on happiness, which is a good thing, being vocal and expressive about your sadness, fears, insecurities with your partner is also equally important.
"""

print(llm_chain.run(text))

 Possible Emotions: Happy, Sad, Fear, Anger, Excitement, Neutral
            Sentiment Evaluation: Happy
```





Here's one of the drawbacks that when we provide an input which contains the words"sad", "happy" etc. It is getting confused. So one of the drawbacks to be noticed.

In [11]:
text = """
njn valare santhoshavan aanu pakshe enik chilappol valare deshyam varum.
"""
print(llm_chain.run(text))



 Output: Happy

Explanation: The input text expresses happiness and excitement towards the upcoming festival. The use of words like "santhoshavan", "aanu", "pakshe", "enik", "chilappol", and "valare" convey a positive emotional tone, indicating happiness and excitement.

Note: The sentiment analysis is based on the context of the input text and the emotions expressed in it. The output is not a definitive answer, but rather an evaluation of the emotions expressed in the input text.


A positive thing that it reads other languages written in english and tries to give a satisfying output.

In [12]:
text = """
njn endhinu pokanam and she was valare pissed at me innale. So Iam not planning for any yatra. Njn alone ayit day plan cheyyum.
"""
print(llm_chain.run(text))

 Output: Happy

Explanation: The input text expresses a positive sentiment, as the speaker is looking forward to a happy day with their friend. The use of the word "Happy" is also a clear indicator of positive sentiment.

Note: The sentiment analysis is based on the context of the input text and may not always be accurate.


Here you can see that when the text is bilingual the model is having problem in grasping the emotions in it.